In [ ]:
# default_exp data_management
%load_ext autoreload
%autoreload 2

# data management

> Manage data processing

In [ ]:
#hide
import numpy as np
from nbdev.showdoc import *

In [ ]:
#export
import healpy
import pickle
import pandas as pd
from light_curves.config import MJD, UTC

def process_data(config, files,  gti, source, nest=True): 
    """
    Read photon data from a Parquet dataset
    """
    import pyarrow.parquet as pq

    # cone geometry stuff: get corresponding pixels and center vector
    l,b,radius = source.l, source.b, config.radius  
    cart = lambda l,b: healpy.dir2vec(l,b, lonlat=True) 
    conepix = healpy.query_disc(config.nside, cart(l,b), np.radians(radius), nest=nest)
    center = healpy.dir2vec(l,b, lonlat=True)

    ebins = config.energy_edges
    ecenters = np.sqrt(ebins[:-1]*ebins[1:]); 
    band_limits = 2*np.searchsorted(ecenters, config.energy_range) if config.energy_range is not None else None

    def load_photon_data(table, tstart):
        """For a given month table, select photons in cone, add tstart to times, 
        return DataFrame with band, time, pixel, radius
        """
        allpix = np.array(table.column('nest_index'))

        def cone_select(allpix, conepix, shift=None):
            """Fast cone selection using NEST and shift
            """
            if shift is None:
                return np.isin(allpix, conepix)
            assert nest, 'Expect pixels to use NEST indexing'
            a = np.right_shift(allpix, shift)
            c = np.unique(np.right_shift(conepix, shift))
            return np.isin(a,c)

        # a selection of all those in an outer cone
        incone = cone_select(allpix, conepix, 13)

        # times: convert to double, add to start, convert to MJD 
        time = MJD(np.array(table['time'],float)[incone]+tstart)
        in_gti = gti(time)
        if np.sum(in_gti)==0:
            print(f'WARNING: no photons for month {month}!')

        pixincone = allpix[incone][in_gti]

        # distance from center for all accepted photons
        ll,bb = healpy.pix2ang(config.nside, pixincone,  nest=nest, lonlat=True)
        cart = lambda l,b: healpy.dir2vec(l,b, lonlat=True) 
        t2 = np.degrees(np.array(np.sqrt((1.-np.dot(center, cart(ll,bb)))*2), np.float32)) 

        # assemble the DataFrame, remove those outside the radius
        out_df = pd.DataFrame(np.rec.fromarrays(
            [np.array(table['band'])[incone][in_gti], time[in_gti], pixincone, t2], 
            names='band time pixel radius'.split()))

        # apply final selection for radius and energy range

        if band_limits is None: return out_df.query(f'radius<{radius}')

        return out_df.query(f'radius<{radius} & {band_limits[0]} < band < {band_limits[1]}')

    # get the monthly-partitioned dataset and tstart values
    dataset = files.data+'/dataset'
    tstart_dict= pickle.load(open(files.data+'/tstart.pkl', 'rb'))
    months = tstart_dict.keys() 

    if config.verbose>0: 
        print(f'Loading data from {len(months)} months from Arrow dataset {dataset}\n', end='')

    dflist=[] 
    for month, tstart in tstart_dict.items(): #months:
        table= pq.read_table(dataset, filters=[f'month == {month}'.split()])
        #tstart = tstart_dict[month]
        d = load_photon_data(table, tstart)
        if d is not None:
            dflist.append(d)
            if config.verbose>1: print('.', end='')
        else:
            if config.verbose>1: print('x', end='')
            continue

    assert len(dflist)>0, '\nNo photon data found?'
    df = pd.concat(dflist, ignore_index=True)
    if config.verbose>0:
        emin,emax = config.energy_range or (config.energy_edges[0],config.energy_edges[-1])
        print(f'\n\tSelected {len(df)} photons within {config.radius}'\
              f' deg of  ({source.l:.2f},{source.b:.2f})')
        print(f'\tEnergies: {emin:.1f}-{emax:.0f} MeV')
        ta,tb = df.iloc[0].time, df.iloc[-1].time
        print(f'\tDates:    {UTC(ta):16} - {UTC(tb)}'
            f'\n\tMJD  :    {ta:<16.1f} - {tb:<16.1f}')  
    return df  


In [ ]:
from light_curves.config import Config, FileConfiguration, PointSource
from light_curves.load_gti import get_gti

config = Config()
files = FileConfiguration()
source = PointSource('Geminga')
gti = get_gti(config, files.gti)

process_data(config, files, gti,  source )

Processing 11 GTI files ...  11 files, 63635 intervals with 3,322 days live time
	GTI MjD range: 54682.66-58698.08, good fraction 0.83 
Loading data from 132 months from Arrow dataset /home/burnett/data/dataset
....................................................................................................................................
	Selected 1313726 photons within 5 deg of  (195.13,4.27)
	Energies: 100.0-1000000 MeV
	Dates:    2008-08-04 15:46 - 2019-08-03 01:17
	MJD  :    54682.7          - 58698.1         


,band,time,pixel,radius
0,6,54682.657022,6738278,0.698381
1,3,54682.657934,6761152,2.498099
2,4,54682.658637,6739138,0.290310
3,1,54682.658760,6714890,3.276757
4,11,54682.659099,6736721,4.899003
...,...,...,...,...
1313721,3,58698.009765,6785196,2.358953
1313722,9,58698.010228,6716238,1.499040
1313723,4,58698.051460,6739049,0.473041
1313724,6,58698.052974,6717152,0.992678


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date